https://www.kaggle.com/cdeotte/part-2-rapids-tfidfvectorizer-cv-0-700

# setup

In [2]:
import numpy as np
import pandas as pd

HOME = '/data/git/shopee-product-matching'
pdata = f'{HOME}/data/shopee-product-matching'

# metric

In [3]:
# def getMetric(col):
#     def f1score(row):
#         n = len( np.intersect1d(row.target,row[col]) )
#         return 2*n / (len(row.target)+len(row[col]))
#     return f1score

def f1(x,y):
    n = len(np.intersect1d(x,y))
    return 2*n / (len(x)+len(y))

def f1s(xs,ys):
    return (f1(x,y) for x,y in zip(targets, preds))

def meanf1(xs,ys):
    return np.mean(list(f1s(xs, ys)))

# data and target

In [4]:
df = pd.read_csv(f'{pdata}/train.csv')
df.head(2)

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045


In [5]:
grp2ids = df.groupby('label_group').posting_id.agg('unique').to_dict()
targets = df.label_group.map(grp2ids)

In [6]:
targets[:2]

0     [train_129225211, train_2278313361]
1    [train_3386243561, train_3423213080]
Name: label_group, dtype: object

# baselines

## self baseline

In [7]:
preds = df.posting_id.apply(lambda x: [x])
preds[0], targets[0]

(['train_129225211'],
 array(['train_129225211', 'train_2278313361'], dtype=object))

In [8]:
meanf1(preds,targets)

0.4608481791365001

## same hash baseline

In [9]:
hsh2ids = df.groupby('image_phash').posting_id.agg('unique').to_dict()
preds = df.image_phash.map(hsh2ids)

preds[0], targets[0]

(array(['train_129225211'], dtype=object),
 array(['train_129225211', 'train_2278313361'], dtype=object))

In [10]:
meanf1(preds,targets)

0.5530933399168149

## image sims

In [48]:
pids = df.posting_id.values

In [49]:
vsims = np.load(f"{HOME}/data/dev0005/vsims.npy")

In [50]:
sims = vsims

In [51]:
sims.shape

(34250, 34250)

In [52]:
np.argsort(-sims[:5])

array([[    0,  5901, 30265, ..., 10421,  5495, 14340],
       [    1,   469,   194, ..., 16518, 20468,  7717],
       [    2, 19467,  7973, ..., 12183, 27815, 14177],
       [    3, 18753, 26063, ..., 25732, 14340,  7717],
       [    4, 18449, 24039, ..., 12867, 12866, 12865]])

In [53]:
topn = 5
preds_i = np.argsort(-sims[:5])[:,:topn]
preds_i

array([[    0,  5901, 30265,  4211,  4861],
       [    1,   469,   194, 18683, 25199],
       [    2, 19467,  7973, 21623,  3446],
       [    3, 18753, 26063, 33884, 27812],
       [    4, 18449, 24039,  9354, 24142]])

In [54]:
preds = pids[preds_i]
preds

array([['train_129225211', 'train_1474389719', 'train_512297426',
        'train_3419767282', 'train_1815147850'],
       ['train_3386243561', 'train_2120597446', 'train_1816968361',
        'train_3423213080', 'train_3805508898'],
       ['train_2288590299', 'train_2723454438', 'train_326726596',
        'train_2661129964', 'train_1479109088'],
       ['train_2406599165', 'train_1593362411', 'train_4014057720',
        'train_538986069', 'train_2560623727'],
       ['train_3369186413', 'train_921438619', 'train_509297860',
        'train_2890561395', 'train_888879443']], dtype=object)

In [55]:
meanf1(preds,targets[:5])

0.39999999999999997

In [56]:
def get_preds_by_topn(sims, topn):
    preds_i = np.argsort(-sims)[:,:topn]
    preds = pids[preds_i]
    return preds

In [57]:
preds = get_preds_by_topn(sims, 2)

In [58]:
meanf1(preds,targets)

0.5778117924816668

## text sims

In [59]:
tsims = np.load(f"{HOME}/data/dev0005/tsims.npy")

In [60]:
sims = tsims

In [61]:
preds = get_preds_by_topn(sims, 2)

In [62]:
meanf1(preds,targets)

0.5506453022799928

## hash sims

In [63]:
hsims = np.load(f"{HOME}/data/dev0011/hsims.npy")

In [64]:
sims = hsims

In [65]:
preds = get_preds_by_topn(sims, 2)

In [66]:
meanf1(preds,targets)

0.4820275227519535